# Packages

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

from PyDSSAT import SimpleCrop

# Parameters

In [2]:
START_DATE = '2020-02-26'
END_DATE = '2020-07-03'

In [3]:
START_DATE = pd.Timestamp(START_DATE)
END_DATE = pd.Timestamp(END_DATE)

print(f'Cultivation period: {(END_DATE - START_DATE).days} days')
print(f'Starting DOY: {START_DATE.dayofyear}')

Cultivation period: 128 days
Starting DOY: 57


# Preparing weather data

In [4]:
env_df = pd.read_csv('../ch2-1_weighing/results/2020_S/SW2_greenhouse.csv', index_col='Unnamed: 0')
env_df.index = pd.DatetimeIndex(env_df.index)
env_df.loc[env_df['rad'] < 5, 'rad'] = 0

In [5]:
rad_df = env_df['rad'].resample('H').mean()*2.54
rad_df = rad_df.resample('D').sum()*3600/1e6

In [6]:
temp_max_df = env_df['temp'].resample('D').max()
temp_min_df = env_df['temp'].resample('D').min()

In [7]:
weather_df = pd.concat([rad_df, temp_max_df, temp_min_df], axis=1)

In [8]:
weather_df['rain'] = 0
weather_df['par'] = weather_df['rad']/2

In [9]:
weather_df.columns = ['rad', 't_max', 't_min', 'rain', 'par']

In [10]:
doy_index = weather_df.index.strftime('%y') + weather_df.index.dayofyear.map('{:03d}'.format)
doy_index = doy_index.astype('int64')

In [11]:
weather_df.index = doy_index

In [12]:
weather_df.to_csv('./results/weather.csv')

# Preparing soil data

In [13]:
soil_values = np.array([0.06, 0.17, 0.28, 145.000, 0.10, 55.00, 246.5]).T
soil_columns = ['WPp', 'FCp', 'STp', 'DP', 'DRNp', 'CN', 'SWC']

In [14]:
soil_df = pd.DataFrame(soil_values, index=soil_columns, columns=['value']).T

In [15]:
soil_df.to_csv('./results/soil.csv')